In [1]:
import kts
kts.config.storage_path = "/mnt/tenTB/avalur/santander_storage/"

In [2]:
from kts.validation.validator import *
from kts.validation.split import *
from kts.zoo.binary_classification import *
from kts.feature import stl
from kts.feature.decorators import *
from kts.optimization.optimizer import *
from kts.feature.storage import feature_list as features
from kts.feature.storage import FeatureSet
from kts.validation.experiment import experiment_list as experiments
from kts.storage.cache_utils import clear_storage

In [3]:
kts.features.define_in_scope(globals())

In [4]:
import numpy as np
import pandas as pd
import feather
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm as tm
from tqdm import trange

In [5]:
%%time
train = feather.read_dataframe('../input/train.fth')
test = feather.read_dataframe('../input/test.fth')
oof = train[['ID_code', 'target']]
oof['predict'] = 0
predictions = test[['ID_code']]
val_aucs = []
feature_importance_df = pd.DataFrame()
# train.set_index('ID_code', inplace=True)
# test.set_index('ID_code', inplace=True)

CPU times: user 270 ms, sys: 676 ms, total: 947 ms
Wall time: 11.2 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
def augment(x, y, t=2):
    xs, xn = [], []
    for i in range(t):
        mask = y > 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:, c] = x1[ids][:, c]
        xs.append(x1)

    for i in range(t // 2):
        mask = y == 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:, c] = x1[ids][:, c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [20]:
random_state = 37
lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state,
    "n_jobs": 4
}

In [8]:
fs_pe = FeatureSet(
    [peak_encoding], 
    df_input=train, 
    target_column='target'
)
fs_pe_test = FeatureSet(
    [peak_encoding],
    df_input=test
)

In [9]:
train.shape, test.shape

((200000, 202), (200000, 201))

In [10]:
train_pe = pd.concat([train, fs_pe.df], axis=1, sort=False)
test_pe = pd.concat([test, fs_pe_test.df], axis=1, sort=False)
train_pe.shape, test_pe.shape

((200000, 289), (200000, 288))

In [11]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [12]:
train_pe.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,pe_var_179,pe_var_180,pe_var_184,pe_var_188,pe_var_190,pe_var_191,pe_var_194,pe_var_196,pe_var_197,pe_var_198
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,False,False,False,False,False,False,False,False,False,False
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,True,False,False,False,False,False,False,False,False,False
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,False,False,False,False,False,False,False,False,False,False
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,False,False,False,False,False,False,False,False,False,False
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,False,False,False,False,False,False,False,False,False,False


In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [14]:
# types = {i: np.dtype(float) if i <=203 else np.dtype(bool) for i in range(X_t.shape[1])}

In [17]:
features = [col for col in train_pe.columns if col not in ['target', 'ID_code']]
X_test = test_pe[features].values

In [21]:
df_train = train_pe
df_test = test_pe

import lightgbm as lgb
from sklearn.metrics import roc_auc_score

for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, df_train['target'])):
# for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train.head(100), df_train['target'][:100])):
    X_train, y_train = df_train.iloc[trn_idx][features], df_train.iloc[trn_idx]['target']
    X_valid, y_valid = df_train.iloc[val_idx][features], df_train.iloc[val_idx]['target']
    
    N = 5
    p_valid,yp = 0,0

    for i in range(N):
        X_t, y_t = augment(X_train.values, y_train.values)
        X_t = pd.DataFrame(X_t)
        types = {i: np.dtype(float) if i<200 else np.dtype(bool) for i in range(X_t.shape[1])}
        X_t = X_t.astype(types)
        X_t = X_t.add_prefix('var_')
    
        trn_data = lgb.Dataset(X_t, label=y_t)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        lgb_clf = lgb.train(lgb_params,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    oof['predict'][val_idx] = p_valid / N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold+1)] = yp / N

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.910384	valid_1's auc: 0.863702
[2000]	training's auc: 0.923064	valid_1's auc: 0.878049
[3000]	training's auc: 0.930718	valid_1's auc: 0.886919
[4000]	training's auc: 0.935402	valid_1's auc: 0.892051
[5000]	training's auc: 0.93884	valid_1's auc: 0.895424
[6000]	training's auc: 0.941397	valid_1's auc: 0.897557
[7000]	training's auc: 0.943588	valid_1's auc: 0.89899
[8000]	training's auc: 0.945498	valid_1's auc: 0.89984
[9000]	training's auc: 0.94724	valid_1's auc: 0.900538
[10000]	training's auc: 0.948907	valid_1's auc: 0.901005
[11000]	training's auc: 0.950531	valid_1's auc: 0.901375
[12000]	training's auc: 0.9521	valid_1's auc: 0.901608
[13000]	training's auc: 0.953632	valid_1's auc: 0.901663
[14000]	training's auc: 0.955121	valid_1's auc: 0.901761
[15000]	training's auc: 0.956571	valid_1's auc: 0.901844
[16000]	training's auc: 0.958016	valid_1's auc: 0.901852
[17000]	training's auc: 0.959405	valid_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: S

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.910485	valid_1's auc: 0.863851
[2000]	training's auc: 0.923159	valid_1's auc: 0.877764
[3000]	training's auc: 0.930803	valid_1's auc: 0.885736
[4000]	training's auc: 0.935575	valid_1's auc: 0.890489
[5000]	training's auc: 0.938972	valid_1's auc: 0.893456
[6000]	training's auc: 0.941506	valid_1's auc: 0.895427
[7000]	training's auc: 0.943612	valid_1's auc: 0.896626
[8000]	training's auc: 0.945498	valid_1's auc: 0.89732
[9000]	training's auc: 0.947243	valid_1's auc: 0.897828
[10000]	training's auc: 0.948907	valid_1's auc: 0.898161
[11000]	training's auc: 0.950512	valid_1's auc: 0.898433
[12000]	training's auc: 0.952091	valid_1's auc: 0.898649
[13000]	training's auc: 0.953613	valid_1's auc: 0.898746
[14000]	training's auc: 0.955095	valid_1's auc: 0.898771
[15000]	training's auc: 0.956545	valid_1's auc: 0.898756
[16000]	training's auc: 0.957986	valid_1's auc: 0.89875
[17000]	training's auc: 0.959365	va

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: S

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.910844	valid_1's auc: 0.862176
[2000]	training's auc: 0.923764	valid_1's auc: 0.876165
[3000]	training's auc: 0.931406	valid_1's auc: 0.884237
[4000]	training's auc: 0.936211	valid_1's auc: 0.889001
[5000]	training's auc: 0.939579	valid_1's auc: 0.891776
[6000]	training's auc: 0.942134	valid_1's auc: 0.893711
[7000]	training's auc: 0.944298	valid_1's auc: 0.894987
[8000]	training's auc: 0.946219	valid_1's auc: 0.895932
[9000]	training's auc: 0.947955	valid_1's auc: 0.896461
[10000]	training's auc: 0.949583	valid_1's auc: 0.896746
[11000]	training's auc: 0.9512	valid_1's auc: 0.896998
[12000]	training's auc: 0.952745	valid_1's auc: 0.897121
[13000]	training's auc: 0.954268	valid_1's auc: 0.897236
[14000]	training's auc: 0.955739	valid_1's auc: 0.897429
[15000]	training's auc: 0.957209	valid_1's auc: 0.897566
[16000]	training's auc: 0.958624	valid_1's auc: 0.89752
[17000]	training's auc: 0.959997	val

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: S

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.90956	valid_1's auc: 0.863505
[2000]	training's auc: 0.922534	valid_1's auc: 0.877961
[3000]	training's auc: 0.930353	valid_1's auc: 0.886397
[4000]	training's auc: 0.935195	valid_1's auc: 0.891607
[5000]	training's auc: 0.938592	valid_1's auc: 0.894717
[6000]	training's auc: 0.941163	valid_1's auc: 0.896734
[7000]	training's auc: 0.943287	valid_1's auc: 0.897924
[8000]	training's auc: 0.945171	valid_1's auc: 0.898784
[9000]	training's auc: 0.946936	valid_1's auc: 0.899465
[10000]	training's auc: 0.948636	valid_1's auc: 0.899794
[11000]	training's auc: 0.950253	valid_1's auc: 0.900071
[12000]	training's auc: 0.951822	valid_1's auc: 0.900307
[13000]	training's auc: 0.953347	valid_1's auc: 0.900501
[14000]	training's auc: 0.954838	valid_1's auc: 0.900625
[15000]	training's auc: 0.956288	valid_1's auc: 0.900632
[16000]	training's auc: 0.957708	valid_1's auc: 0.900672
[17000]	training's auc: 0.95912	va

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: S

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.911273	valid_1's auc: 0.859238
[2000]	training's auc: 0.923878	valid_1's auc: 0.873142
[3000]	training's auc: 0.931505	valid_1's auc: 0.881213
[4000]	training's auc: 0.936257	valid_1's auc: 0.886312
[5000]	training's auc: 0.939647	valid_1's auc: 0.889488
[6000]	training's auc: 0.942145	valid_1's auc: 0.891561
[7000]	training's auc: 0.944291	valid_1's auc: 0.892817
[8000]	training's auc: 0.946195	valid_1's auc: 0.893658
[9000]	training's auc: 0.947914	valid_1's auc: 0.894122
[10000]	training's auc: 0.949585	valid_1's auc: 0.894482
[11000]	training's auc: 0.951197	valid_1's auc: 0.894606
[12000]	training's auc: 0.952751	valid_1's auc: 0.894833
[13000]	training's auc: 0.954279	valid_1's auc: 0.894934
[14000]	training's auc: 0.955736	valid_1's auc: 0.895007
[15000]	training's auc: 0.957197	valid_1's auc: 0.89498
[16000]	training's auc: 0.95863	valid_1's auc: 0.894994
[17000]	training's auc: 0.960015	va

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: S

In [22]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.899342191, std: 0.002395204. All auc: 0.899346523.


In [24]:
# submission
model = 'lgb_aug_pe'
predictions['target'] = np.mean(
    predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv(f'../output/{model}_all_predictions.csv', index=None)
sub_df = pd.DataFrame({'ID_code':df_test['ID_code'].values})
sub_df['target'] = predictions['target']
sub_df.to_csv(f'../output/{model}_submission.csv', index=False)
oof.to_csv(f'../output/{model}_oof.csv', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
